# Load Libraries

The popular shared notebooks are only using the first row of data for each eeg-id.

I believe this is an error and a mis-interpertation of the overview.   

This notebook shows that valuable information is contained when using all rows per eeg_id.

In [ ]:
import os
import pandas as pd, numpy as np
import matplotlib.pyplot as plt



# Load Train Data

In [ ]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('df shape:', df.shape )
print('Targets', list(TARGETS))
df = df.sort_values(by=['patient_id', 'eeg_id', 'eeg_sub_id'])
df

# Create Row Accuracy

In 2003 when I was trained on the 6 sigma process, a standard starting point was to evaluate the measurement system.   Multiple readings on the same 'part' were made.


Multiple rows of data are available for many of the eeg_id.  Lets use those looking at how often agreement for the 'expert_consensus' existed for each row.



In [ ]:
# Adding a new column 'total_evaluators' that sums up the six specified columns
df['total_evaluators'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].sum(axis=1)

df.sample(25)  # Display the DataFrame with the new column

In [ ]:
import matplotlib.pyplot as plt

# Plotting a histogram for the 'total_evaluators' column in the 'df' DataFrame

plt.figure(figsize=(10, 6))
plt.hist(df['total_evaluators'], bins=10, color='blue', edgecolor='black')
plt.title('Histogram of Total Evaluators')
plt.xlabel('Total Evaluators')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

It would seem that the training data might be aggregrated from at least two different studies with different number of persons doing the evaluations.   

With more evaluators it is likely data that should have more weight if there is good consensus.

In [ ]:
# Modifying the previous code to add an additional column 'consensus_column' to 'df'

# Finding the column with the largest number for each row and storing the value in 'consensus'
df['consensus'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].max(axis=1)

# Identifying the column name that corresponds to the max value for each row
df['consensus_column'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].idxmax(axis=1)

df.head()  # Display the DataFrame with the new columns



df.sample(25)

In [ ]:
# create a new column that shows the percentage agreement
df['row_agreement'] = df['consensus']/df['total_evaluators']
df.sample(25)

In [ ]:
df.to_csv('row_agreement.csv', index = False)

In [ ]:
# Plotting a histogram for the 'row_agreement' column

import numpy as np


# Now, plotting the histogram for 'row_agreement'
plt.figure(figsize=(10, 6))
plt.hist(df['row_agreement'], bins=10, color='green', edgecolor='black')
plt.title('Histogram of Row Agreement')
plt.xlabel('Row Agreement')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

This looks a bit odd.  Again kind of thinking that multiple studies put togeather for for train data, with some of them having poor agreement among evalators.   Many of the 1.0 ratings are for rows with small number of evalators.

Rows that have very low agreement on the consensus are a problem to be addressed - not sure what's is best?

In [ ]:
# Plotting an XY plot for 'row_agreement' vs 'total_evaluators'

plt.figure(figsize=(10, 6))
plt.scatter(df['row_agreement'], df['total_evaluators'], color='purple', edgecolor='black')
plt.title('XY Plot of Row Agreement vs Total Evaluators')
plt.xlabel('Row Agreement')
plt.ylabel('Total Evaluators')
plt.grid(True)
plt.show()



I am tempted to suggest that each of the 6 seizer types has a different degree of difficulty for evaluators.


In [ ]:
# Assuming 'df' has a mechanism to identify which of the 6 columns ('seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote') is the consensus for each row
# We will generate a plot that shows 'row_agreement' values when each of these columns is the consensus vote

# For demonstration, let's assume 'consensus_column' is a column that indicates which of the 6 columns is the consensus
# This step is for demonstration purposes and should be replaced with your actual method of determining the consensus column
df['consensus_column'] = df[['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']].idxmax(axis=1)

# Now, let's plot 'row_agreement' for each of the 6 columns when they are the consensus
plt.figure(figsize=(12, 8))

for column in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']:
    # Filter the DataFrame for rows where this column is the consensus
    filtered_df = df[df['consensus_column'] == column]
    # Plotting
    plt.scatter(filtered_df['row_agreement'], [column] * len(filtered_df), label=column)

plt.title('Row Agreement for Each Column as Consensus')
plt.xlabel('Row Agreement')
plt.yticks(['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])
plt.ylabel('Consensus Column')
plt.legend()
plt.grid(True)
plt.show()



Little hard to see the visual information with this plot.

Lets revise the plot to distribution curves.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

# Plotting distribution curves for each column
for column in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']:
    # Filter the DataFrame for rows where this column is the consensus
    filtered_df = df[df['consensus_column'] == column]

    # Plotting the distribution curve with clipping the x-axis range
    sns.kdeplot(filtered_df['row_agreement'], label=column, clip=(0, 1.0))

plt.title('Distribution of EEG_ID Agreement for Each Column as Consensus')
plt.xlabel('Row Agreement')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


OK - this plot a little easier for me to interpert.

1.  Some eeg-id have very poor concensus at 20% agreement or less.   Might not want to use this data ?
2.  seize_vote seems the easiest to rate 
3.  gpd_vote would seem to be the hardest.  
4.  Again an appearnce that at least two different studies were aggregated to form our train data.


##  eeg

Lets repeat this data analysis by eeg_id.  There are a number of eeg_id that have multiple rows of data.

First lets see the distribution of evaluations per egg_id

In [ ]:
# Assuming 'eeg_id' is a column in the 'df' DataFrame
# We will generate a histogram that shows the count of rows for each unique 'eeg_id'

# Counting the number of rows for each unique 'eeg_id'
# Adding the 'eeg_id_counts' to the DataFrame 'df'
# This will map each 'eeg_id' in 'df' to its count

# First, create a Series with 'eeg_id' as the index and the counts as values
eeg_id_counts = df['eeg_id'].value_counts()

# Mapping each 'eeg_id' in 'df' to its count
df['eeg_id_counts'] = df['eeg_id'].map(eeg_id_counts)




# Plotting the histogram
plt.figure(figsize=(12, 6))
plt.hist(eeg_id_counts, bins=100, color='orange', edgecolor='black')
plt.title('Histogram of Row Counts for Each Unique EEG ID')
plt.xlabel('Number of Rows per EEG ID')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()



Looks like a patient or two hooked up to an eeg on a single session for long time.    

The current popular shared notebooks are looking at only the first row per unique eeg_id.  That's how they go from 100K to 17K rows of data.

I think using only the first is leaving information on the table, but not sure I want to include 700 rows for a single eeg_id - hmmm   what to do?

In [ ]:
df.head(25)  # Display the first few rows of the DataFrame to show the new column


In [ ]:
row_agreement_agg = df.groupby('eeg_id')['consensus'].agg('sum')

# Mapping this aggregated value back to each row in 'df'
df['row_consensus_agg'] = df['eeg_id'].map(row_agreement_agg)


row_evaluators_agg = df.groupby('eeg_id')['total_evaluators'].agg('sum')

# Mapping this aggregated value back to each row in 'df'
df['row_evaluators_agg'] = df['eeg_id'].map(row_evaluators_agg)

df['eeg_agreement'] = df['row_consensus_agg']/df['row_evaluators_agg']

df.head(25)  # Display the first few rows of the DataFrame to show the new column



In [ ]:
df.to_csv('eeg_agreement.csv', index = False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

# Plotting distribution curves for each column
for column in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']:
    # Filter the DataFrame for rows where this column is the consensus
    filtered_df = df[df['consensus_column'] == column]

    # Plotting the distribution curve with clipping the x-axis range
    sns.kdeplot(filtered_df['eeg_agreement'], label=column, clip=(0, 1.0))

plt.title('Distribution of EEG_ID Agreement for Each Column as Consensus')
plt.xlabel('Row Agreement')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()



Row and eeg_id appear to have similiar plots.  

Expanding on this theme - lets look at patient_id

In [ ]:
# Assuming 'eeg_id' is a column in the 'df' DataFrame
# We will generate a histogram that shows the count of rows for each unique 'eeg_id'

# Counting the number of rows for each unique 'eeg_id'
# Adding the 'eeg_id_counts' to the DataFrame 'df'
# This will map each 'eeg_id' in 'df' to its count

# First, create a Series with 'eeg_id' as the index and the counts as values
patient_id_counts = df['patient_id'].value_counts()

# Mapping each 'eeg_id' in 'df' to its count
df['patient_id_counts'] = df['patient_id'].map(patient_id_counts)




# Plotting the histogram
plt.figure(figsize=(12, 6))
plt.hist(eeg_id_counts, bins=100, color='orange', edgecolor='black')
plt.title('Histogram of Row Counts for Each Unique Patient ID')
plt.xlabel('Number of Rows per patient ID')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Since a single patient is in a single eeg_id the distributions should be pretty similiar.

In [ ]:


row_agreement_agg = df.groupby('patient_id')['consensus'].agg('sum')

# Mapping this aggregated value back to each row in 'df'
df['patient_consensus_agg'] = df['patient_id'].map(row_agreement_agg)


row_evaluators_agg = df.groupby('patient_id')['total_evaluators'].agg('sum')

# Mapping this aggregated value back to each row in 'df'
df['patient_evaluators_agg'] = df['patient_id'].map(row_evaluators_agg)

df['patient_agreement'] = df['patient_consensus_agg']/df['patient_evaluators_agg']

df.sample(25)

In [ ]:
plt.figure(figsize=(12, 8))

# Plotting distribution curves for each column
for column in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']:
    # Filter the DataFrame for rows where this column is the consensus
    filtered_df = df[df['consensus_column'] == column]

    # Plotting the distribution curve with clipping the x-axis range
    sns.kdeplot(filtered_df['patient_agreement'], label=column, clip=(0, 1.0))

plt.title('Distribution of Patient Agreement for Each Column as Consensus')
plt.xlabel('Row Agreement')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


Couple of ways to look at this plot.

Seizure_vote - patients with this condition have consistent eeg's
OR
easy for evalators to spot and agree.

gpd_vote - patients with this condition don't show the issue over time - it comes and goes
OR
very hard for evalators to spot - the plot suggest we have no eeg for this type where all the evalators agreed 


One Conclusion - the shared notebooks that use only the 'first' are missing valuabe information.

As an initial use of this information I think I will use the patient agreement values as weights in my fork of Chris's catboost model.




In [ ]:
df

In [ ]:
df.to_csv('train_upgraded.csv', index = False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['patient_agreement'], bins=100, color='green', edgecolor='black')
plt.title('Histogram of Patient Agreement')
plt.xlabel('Row Agreement')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()